In [31]:
from search import *
from game import *
from util import *
from pacman import *
import util, layout
import sys, types, time, random, os



In [47]:
# tiny genetic programming by © moshe sipper, www.moshesipper.com
from random import random, randint, seed
from statistics import mean
from copy import deepcopy

POP_SIZE        = 60   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 5    # maximal initial random tree depth
GENERATIONS     = 250  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 



def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y

"FUNCOES ( sao as operacoes executadas nos nós das arvores)-> add, sub, mul, div (x/0 = 1)*"
"TERMINAIS ( sao os tipos possiveis de nohs folha )"
'''
    TERMINAIS ( sao os tipos possiveis de nohs folha )
        -> GAMESTATE_X, onde 0 <= X < Num. de variaveis de estado do jogo
        -> CONSTANTS ( Uns numeros constantes para algumas operações, para facilitar o calculo numerico (?) )
        
    
'''




FUNCTIONS = [add, sub, mul]
TERMINALS = ['actionScore', 'eatFood', 'minGhostDist', 'distToNextPill', -1.0, 0.5, 1.0, 2.0] # 0.0,  -0.5,

STATE_VARS = ['actionScore', 'eatFood', 'minGhostDist', 'distToNextPill']
" actionScore: scoreFuturo - scoreAtual "
" eatFood: numFoodAtual - numFoodFuturo"
" minGhostDist: sqrt((pacmanY - pacmanX)**2 + (ghostY - ghost)X**2) "
" distToNextPill: sqrt((pacmanY - pacmanX)**2 + (ghostY - ghost)X**2) ?????????? "


class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left
        self.right = right
        
    def setLeft(self, tree):
        self.left = tree
        
    def setRight(self, tree):
        self.right = tree
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")

    def computeTree(self, stateVars): 
        if (self.data in FUNCTIONS): 
            return self.data(self.left.computeTree(stateVars), self.right.computeTree(stateVars))
        elif (self.data in STATE_VARS): 
            return stateVars[self.data]
        else: return self.data
        
    def size(self): # tree size in nodes
        if self.data in TERMINALS: return 1
        l = self.left.size()  if self.left  else 0
        r = self.right.size() if self.right else 0
        return 1 + l + r
    
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        if self.data in FUNCTIONS:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)
            
    def getSubtree(self):
        t = GPTree()
        t.data = self.data
        if self.left:  t.left  = self.left.getSubtree()
        if self.right: t.right = self.right.getSubtree()
        return t
    
    def scan_tree(self, count, second): # note: count is list, so it's passed "by reference"
        count[0] -= 1            
        if count[0] <= 1: 
            if not second: # return subtree rooted here
                return self.getSubtree()
            else: # glue subtree here
                self.data  = second.data
                self.left  = second.left
                self.right = second.right
        else:  
            ret = None              
            if self.left  and count[0] > 1: ret = self.left.scan_tree(count, second)  
            if self.right and count[0] > 1: ret = self.right.scan_tree(count, second)  
            return ret
        

In [54]:
tree = GPTree()
tree.random_tree(True, 2)
tree.print_tree()
thisdict = {
    STATE_VARS[0]: 1.0,
    STATE_VARS[1]: 2.0,
    STATE_VARS[2]: 3.0,
    STATE_VARS[3]: 4.0,
}
print(tree.computeTree(thisdict))

add
   add
      0.5
      1.0
   add
      eatFood
      1.0
4.5


In [34]:
" Criar populacao inicial"
''' Para a criação da população inicial, utilizamos o método RandomTree() que gera árvores randomicamente dentro 
    do domínio de operações fundamentais (add, sub, mul) 
        e dos terminais definidos (variaveis de estado e constantes)
'''

' Para a cria\xc3\xa7\xc3\xa3o da popula\xc3\xa7\xc3\xa3o inicial, utilizamos o m\xc3\xa9todo RandomTree() que gera \xc3\xa1rvores randomicamente dentro \n    do dom\xc3\xadnio de opera\xc3\xa7\xc3\xb5es fundamentais (add, sub, mul) \n        e dos terminais definidos (variaveis de estado e constantes)\n'

In [35]:
POP_SIZE = 32
N_TRAIN = 4


def initialPopulation():
    population = []
    for _ in range(32):
        tree = GPTree()
        tree.random_tree(True, 2)
        population.append(tree)
    return population

In [36]:
" FITNESS "

# stateVars -> dictionary  ex stateVars['distX'] = distXGameStateVariable
def getFitness(individuo, stateVars):
    return individuo.computeTree(stateVars)

def getPopFitness(population, stateVars):
    return np.array([getFitness(population[i], stateVars) for i in range(0,len(population))])

    
    

" SELEÇÂO "
'''Para algotimo de seleção, adotaremos inicialmente o método de seleção por Torneio, na ideia de
    aumentar as randomicidade do modelo.
'''
def tournament(population, stateVars):
    indexes = np.random.choice(range(0,len(population)),4,False)
    return population[indexes[np.argmax(getPopFitness(population[indexes], stateVars))]]

def getParent(population, stateVars):
    return tournament(population, stateVars)


" CROSSOVER "
''' Para a reprodução dos pais, utilizamos uma função de crossover para árvores, 
'''

def crossover(treeA, treeB): # xo 2 trees at random nodes
#     if random() < XO_RATE:
#         geneA = treeA.scan_tree([randint(1, treeA.size())], None) # 1st random subtree
#         geneB = treeB.scan_tree([randint(1, treeB.size())], None) # 2nd random subtree
#         treeA.scan_tree([randint(1, treeA.size())], geneB) # 2nd subtree "glued" inside 1st tree
#         treeB.scan_tree([randint(1, treeB.size())], geneA) # 1st subtree "glued" inside 2nd tree
        
    child1 = treeA.getSubtree()
    child2 = treeB.getSubtree()
    
    aux = child1.right.getSubtree()
    aux.print_tree()
    child1.right = child2.left.getSubtree()
    child2.left = aux
    
    return child1, child2
    
            
" Loop de Reproducao"









' Loop de Reproducao'

In [74]:
tree = GPTree()
tree.random_tree(True, 3)
tree.print_tree()

tree2 = GPTree()
tree2.random_tree(True, 3)
tree2.print_tree()

c1, c2 = crossover(tree, tree2)
c1.print_tree()
c2.print_tree()

sub
   mul
      1.0
      mul
         -1.0
         2.0
   add
      mul
         -1.0
         2.0
      sub
         minGhostDist
         1.0
sub
   mul
      distToNextPill
      mul
         2.0
         actionScore
   mul
      -1.0
      add
         minGhostDist
         actionScore
add
   mul
      -1.0
      2.0
   sub
      minGhostDist
      1.0
sub
   mul
      1.0
      mul
         -1.0
         2.0
   mul
      distToNextPill
      mul
         2.0
         actionScore
sub
   add
      mul
         -1.0
         2.0
      sub
         minGhostDist
         1.0
   mul
      -1.0
      add
         minGhostDist
         actionScore


In [77]:
n = Directions.NORTH
s = Directions.SOUTH
e = Directions.EAST
w = Directions.WEST

def manhattanDistance(posA, posB):
    return (abs(posB[0] - posA[0]) + abs(posB[1] - posA[1]))

def calcMinGhostDist(pacmanPosition, ghostsPosition):
    dist = 99999
    for ghost in ghostsPosition:
        if dist > manhattanDistance(pacmanPosition, ghost):
            dist = manhattanDistance(pacmanPosition, ghost)
            
    return dist

def manhattanFoodDistance(posA, posX, posY):
    return (abs(posX - posA[0]) + abs(posY - posA[1]))

def calcMinFoodDist(pacmanPosition, foodGrid):
    dist = 99999
    for i in range(len(foodGrid[:])):
        for j in range(len(foodGrid[0][:])):
            if foodGrid[i][j] and (dist > manhattanFoodDistance(pacmanPosition, i, j)):
                dist = manhattanFoodDistance(pacmanPosition, i, j)

    return dist

class MyGoWestAgent(Agent):
    "An agent that goes West until it can't."
    def __init__(self, tree = None):
        self.tree = tree
        
    def setTree(self, tree):
        selt.tree = tree

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."
        print(state.getLegalPacmanActions())
        print(state.getPacmanState())

        print(state.getPacmanPosition())
        print(state.getNumFood())
        print(state.getScore())
        count = 0
        legalActions = state.getLegalPacmanActions()
        legalActions.remove(Directions.STOP) #remove a opcao Stop
        actionScore = []
        stateVars = {}
        for action in legalActions:
            futureState = state.generatePacmanSuccessor(action)
            futureScore = futureState.getScore() - state.getScore()
            eatFood = state.getNumFood() - futureState.getNumFood()
            ghostDist = calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions())
            foodDist = calcMinFoodDist(futureState.getPacmanPosition(),futureState.getFood())
            stateVars[STATE_VARS[0]] = futureScore
            stateVars[STATE_VARS[1]] = eatFood
            stateVars[STATE_VARS[2]] = ghostDist
            stateVars[STATE_VARS[3]] = foodDist

            treeValue = tree.computeTree(stateVars)
            actionScore.append(treeValue)
            
        
            
        maxIndex = actionScore.index(max(actionScore))
        print("Action Score: ",actionScore)
        action = legalActions[maxIndex]
        
        futureState = state.generatePacmanSuccessor(action)
        print("MinGhostDistance: ", calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions()))
        if action in state.getLegalPacmanActions():
            return action
        else:
            return Directions.STOP

In [78]:
# args = readCommand(['--layout', 'smallClassic', '--pacman', 'GoWestAgent', '-t']) # Get game components based on input
gameLayout = layout.getLayout( 'smallClassic' )

ind = GPTree()
ind.random_tree(True, 3)
ind.print_tree()

pacman = MyGoWestAgent(ind)
# pacman.setTree(ind)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

import textDisplay
textDisplay.SLEEP_TIME = 0.1
gameDisplay = textDisplay.PacmanGraphics()

runGames( gameLayout, pacman, ghosts, gameDisplay, 4, False)

mul
   mul
      minGhostDist
      add
         0.5
         -1.0
   mul
      sub
         2.0
         eatFood
      eatFood
%%%%%%%%%%%%%%%%%%%%
%......%G  G%......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........<.........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

['West', 'Stop', 'East']
Pacman: (x,y)=(9, 1), Stop
(9, 1)
55
0.0
('Action Score: ', [-3.0, -4.0])
('MinGhostDistance: ', 4)
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%.......> .........%
%%%%%%%%%%%%%%%%%%%%
Score: 9

['West', 'Stop', 'East']
Pacman: (x,y)=(8, 1), West
(8, 1)
54
9.0
('Action Score: ', [-5.0, -3.0])
('MinGhostDistance: ', 4.0)
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%GG%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%....... <.........%
%%%%%%%%%%%%%%%%%%%%
Score: 8

['West', 'Stop', 'East']
Pacman: (x,y)=(9, 1), East
(9, 1)
54
8.0
('Action Score: ', [-3.0, -2.0])
('MinGhostDistance: ', 3.0)
%%%%%%%%%%%%%%%%%%%%
%

['West', 'Stop', 'East']
Pacman: (x,y)=(10, 1), East
(10, 1)
34
181.0
('Action Score: ', [-3.0, -3.0])
('MinGhostDistance: ', 4.0)
%%%%%%%%%%%%%%%%%%%%
%    ..%  G %......%
% %%   %%  %%...%%.%
% %o.% ......G%.o%.%
% %%.% %%%%%%.%.%%.%
%        > ........%
%%%%%%%%%%%%%%%%%%%%
Score: 180

['West', 'Stop', 'East']
Pacman: (x,y)=(9, 1), West
(9, 1)
34
180.0
('Action Score: ', [-5.0, -3.0])
('MinGhostDistance: ', 4.0)
%%%%%%%%%%%%%%%%%%%%
%    ..% G  %......%
% %%   %%  %%...%%.%
% %o.% .......%.o%.%
% %%.% %%%%%%G%.%%.%
%         <........%
%%%%%%%%%%%%%%%%%%%%
Score: 179

['West', 'Stop', 'East']
Pacman: (x,y)=(10, 1), East
(10, 1)
34
179.0
('Action Score: ', [-3.0, -2.0])
('MinGhostDistance: ', 3.0)
%%%%%%%%%%%%%%%%%%%%
%    ..%G   %......%
% %%   %%  %%...%%.%
% %o.% .......%.o%.%
% %%.% %%%%%%.%.%%.%
%          <.G.....%
%%%%%%%%%%%%%%%%%%%%
Score: 188

['West', 'Stop', 'East']
Pacman: (x,y)=(11, 1), East
(11, 1)
33
188.0
('Action Score: ', [-2.0, 0.0])
('MinGhostDistance: ', 1.0)
%%

['Stop', 'East', 'North', 'South']
Pacman: (x,y)=(6, 3), West
(6, 3)
9
403.0
('Action Score: ', [-2.0, 0.0, -1.0])
('MinGhostDistance: ', 1.0)
%%%%%%%%%%%%%%%%%%%%
%    G.%    %      %
% %%  v%%  %% ..%% %
% %oG%        %.o% %
% %%.% %%%%%%.%.%% %
%                  %
%%%%%%%%%%%%%%%%%%%%
Score: 402

['West', 'Stop', 'North', 'South']
Pacman: (x,y)=(6, 4), North
(6, 4)
9
402.0
('Action Score: ', [0.0, 0.0, -1.0])
('MinGhostDistance: ', 1.0)
%%%%%%%%%%%%%%%%%%%%
%    .G%    %      %
% %% > %%  %% ..%% %
% %o.%        %.o% %
% %%.% %%%%%%.%.%% %
%                  %
%%%%%%%%%%%%%%%%%%%%
Score: 401

['West', 'Stop', 'East', 'North']
Pacman: (x,y)=(5, 4), West
(5, 4)
9
401.0
('Action Score: ', [-1.0, 0.0, 0.0])
('MinGhostDistance: ', 1.0)
%%%%%%%%%%%%%%%%%%%%
%    ..%    %      %
% %%  G%%  %% ..%% %
% %oG%        %.o% %
% %%.% %%%%%%.%.%% %
%                  %
%%%%%%%%%%%%%%%%%%%%
Score: -100

%%%%%%%%%%%%%%%%%%%%
%    ..%    %      %
% %%  G%%  %% ..%% %
% %oG%        %.o% %
% %%.% %%%%

['West', 'Stop', 'East', 'North']
Pacman: (x,y)=(13, 1), East
(13, 1)
50
38.0
('Action Score: ', [-1.0, -1.0, 1.0])
('MinGhostDistance: ', 0.0)
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%.G......%.o%.%
%.%%.%.%%%%%%G%.%%.%
%.......      .....%
%%%%%%%%%%%%%%%%%%%%
Score: -453

Pacman died! Score: -453
%%%%%%%%%%%%%%%%%%%%
%......%G  G%......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........<.........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

['West', 'Stop', 'East']
Pacman: (x,y)=(9, 1), Stop
(9, 1)
55
0.0
('Action Score: ', [-3.0, -4.0])
('MinGhostDistance: ', 4)
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%.......> .........%
%%%%%%%%%%%%%%%%%%%%
Score: 9

['West', 'Stop', 'East']
Pacman: (x,y)=(8, 1), West
(8, 1)
54
9.0
('Action Score: ', [-5.0, -3.0])
('MinGhostDistance: ', 4.0)
%%%%%%%%%%%%%%%%%%%%
%......%  G %......%
%.%%...%% G%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%


[<game.Game instance at 0x7fb346ed56e0>,
 <game.Game instance at 0x7fb346dca500>,
 <game.Game instance at 0x7fb346dff8c0>,
 <game.Game instance at 0x7fb346dbc3c0>]

In [44]:
def trainIndividual(ind, layoutType='smallClassic'):
    gameLayout = layout.getLayout( 'smallClassic' )

    pacman = MyGoWestAgent(ind)

    ghostType = loadAgent('RandomGhost', True)
    ghosts = [ghostType( i+1 ) for i in range( 4 )]

    import textDisplay
    textDisplay.SLEEP_TIME = 0.1
    gameDisplay = textDisplay.PacmanGraphics()

    games = runGames( gameLayout, pacman, ghosts, gameDisplay, 4, False)
    scores = [game.state.getScore() for game in games]
    wins = [game.state.isWin() for game in games]
    winRate = wins.count(True)/ float(len(wins))
    print('Average Score:', sum(scores) / float(len(scores)))
    print('Scores:       ', ', '.join([str(score) for score in scores]))
    print('Win Rate:      %d/%d (%.2f)' % (wins.count(True), len(wins), winRate))
    print('Record:       ', ', '.join([ ['Loss', 'Win'][int(w)] for w in wins]))

In [41]:
def generationLoop():
    population = initialPopulation()
    fitness = np.zeros((1,len(population)))
    count = 1
    print("G",count)
    while count <= 200:
        # train population to get fitness
        
        for i in range(0,len(population)):
            trainIndividual(population[i])
            
        
        children = []
        
        for i in range(0,int(population.shape[0]/2)):
            father = getParent(population)
            mother = getParent(population)
            while(np.array_equal(father, mother)):
                father = getParent(population)
            child1, child2 = crossover(father, mother)
            children.append(child1)
            children.append(child2)
            

#         children = mutation(children,int(len(children)*0.75))#mutation

        #elitism
        fittest_parent = population[np.argmax(getFitness(population))]
        children[np.argmin(getFitness(children))] = fittest_parent

        population = children.copy()
        print(children)

        fitness = getFitness(population)
        print(count, fitness)
        
        count += 1
        
    return population[np.argmax(getFitness(population))]

IndentationError: expected an indented block (<ipython-input-41-904a676ae57c>, line 13)